In [1]:
import gc
import sys
import shutil
from glob import glob
from pathlib import Path
from tempfile import TemporaryDirectory

import daft
import numpy as np
import timm
import torch
from datasets import load_dataset
from humanize import naturalsize
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from loguru import logger

In [2]:
# easy timestamps
logger.remove()
logger.add(sys.stdout, level="INFO")

1

## Compare daft dataframe -> torch dataset vs. plain torch dataset

I am looking at using daft to do image processing in `bedmap` (F.N.A. clipplot forked from PixPlot)

This library runs a timm embedding model against a folder of images on disk.

It could later be extended to images on object store or other sources.
Daft simplifies some of the initial cleanup pipeline, so it would be nice to 

Steps in this notebook:
1. Set variables for test
2. Define function to download some test images
3. Define "embedder" which is a timm pre-trained image model instantiated to do embeddings
4. Define two types of datasets (daft_to_torch and plain_torch)
5. Define embedding pipeline including the dataset creation
6. Do memory profiling with one dataset option

NOTE: This notebook must be run twice to create comparative memory profiles

## Results summary from profiling last cell

Constants for test:
256 images, batch_size 32, model "vit_small_patch14_reg4_dinov2.lvd142m"

### daft_to_torch:
- Peak resident size: 3.24 Gb
- Peak heap size: 2.46 Gb
- Resident size just before first inference: 2.42 Gb
- Images per second: 2.0
- Profile graphs: ./memray-results-summary/2025-03-14_daft-to-torch.html

### plain_torch:
- Peak resident size: 1.89 Gb
- Peak heap size: 1.15 Gb
- Resident size just before first inference: 1.08 Gb
- Images per second: 1.95
- Profile graphs: ./memray-results-summary/2025-03-14_plain-torch.html

### Thoughts
Memory difference seems all due to something during initialization
at first inference call.

### 1. Set variables for test

In [3]:
USE_DAFT: bool = True # else use torch dataset

BATCH_SIZE = 32
MODEL_NAME = "vit_small_patch14_reg4_dinov2.lvd142m"
TEST_DATASET = "kvriza8/microscopy_images"
NUM_TEST_IMAGES = 256

## 2. Define way to download small test dataset 

In [4]:
def dl_hf_images(dataset_name: str = "kvriza8/microscopy_images",
                 dir: Path = None,
                 max_images: int = 64,
                 overwrite: bool = True,
                 format: str = "png") -> None:

    dataset = load_dataset(dataset_name, split="train", streaming=True)
    if overwrite:
        shutil.rmtree(dir, ignore_errors=True)
        dir.mkdir(parents=True, exist_ok=True)

    image_paths = []
    for i, img_row in enumerate(tqdm(iter(dataset), total=max_images)):
        if i >= max_images:
            break
        img = img_row["image"]
        image_paths += [(dir / f"{i}.{format}")]
        img.save(image_paths[-1])

    print(f"Size of images on disk: {naturalsize(sum([p.stat().st_size for p in image_paths]))}")

    del dataset
    gc.collect()

    return None

## 3. Define timm-based embedding model

In [5]:

class Embedder:
    """instantiate pretrained timm model to generate embeddings"""
    def __init__(self, model_name: str, device: torch.device = None):
        self.model_name = model_name
        # choose device and dtype
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if self.device.type == "cuda":
            self.dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        else:
            self.dtype = torch.float32

        # Create and prepare the model
        self.model = timm.create_model(self.model_name, pretrained=True, num_classes=0)
        self.model.to(self.device, memory_format=torch.channels_last)
        self.model.eval()
        self.model = torch.compile(self.model, dynamic=True, mode="reduce-overhead")

        # must resolve config to drop unneeded fields
        cfg = timm.data.resolve_data_config(self.model.pretrained_cfg)
        self.transform = timm.data.create_transform(**cfg)

    @torch.inference_mode()
    def embed(self, batch_imgs: torch.Tensor) -> torch.Tensor:
        """set up input and embed it"""
        batch_imgs = batch_imgs.to(self.device, non_blocking=True, memory_format=torch.channels_last)
        if self.device.type == "cuda":
            with torch.amp.autocast("cuda", dtype=self.dtype):
                return self.model(batch_imgs)
        return self.model(batch_imgs)

## 4. Define two types of datasets

In [6]:
@daft.udf(return_dtype=daft.DataType.python())
class TransformImagesDaft:
    """run timm embedder on an image column"""
    def __init__(self, transform: callable):
        self.transform = transform

    def __call__(self, batch_images) -> list:
        return [self.transform(Image.fromarray(im)) for im in batch_images.to_pylist()]

In [7]:

class TorchImageIterAsDict(Dataset):
    def __init__(self, filelist: list[Path], transform: callable):
        self.filelist = filelist
        self.transform = transform

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx: int):
        image = Image.open(self.filelist[idx]).convert("RGB")
        if self.transform:
            return {"image_transformed": self.transform(image)}
        # return as dict for easy comparison vs. daft
        else:
            return {"image": [image]}


In [8]:
def daft_to_torch_iter_from_glob(image_glob: str, transform: callable):
    """generate a torch image dataset via daft from a glob"""

    images_df = daft.from_glob_path(image_glob)
    images_df = images_df.with_column("image", daft.col("path"
                                    ).url.download().image.decode(
                                        mode="RGB", on_error="null")
                                    )
    images_df = images_df.where(images_df["image"].not_null())
    TransformImForModel = TransformImagesDaft.with_init_args(transform=transform)
    images_df = images_df.with_column("image_transformed", TransformImForModel(daft.col("image"))
                                    ).exclude("image", "num_rows")
    return images_df.to_torch_iter_dataset()

In [9]:
def torch_iter_from_glob(image_glob: str, transform: callable):
    """generate a torch image dataset via daft from a glob"""

    image_list = [Path(p) for p in glob(image_glob)]
    return TorchImageIterAsDict(image_list, transform)

## 5. Embedding computation pipeline including dataset instantiation

In [10]:
def compute_embeddings(model_name: str, images_glob: str, batch_size: int = BATCH_SIZE,
                       dataset_type: str = "plain_torch"
                       ) -> list[np.ndarray]:
    """
    Return a list of embeddings from a glob of images.
    Uses a timm pretrained model to generate embeddings
    """
    logger.info("Instantiating embedding model.")
    embedder = Embedder(model_name=model_name)

    logger.info(f"Creating dataset of type {dataset_type}.")
    if dataset_type == "daft_to_torch":
        dataset = daft_to_torch_iter_from_glob(images_glob, embedder.transform)
    elif dataset_type == "plain_torch":
        dataset = torch_iter_from_glob(images_glob, embedder.transform)
    else:
        raise ValueError("Dataset type must be `daft_to_torch` or `plain_torch`.")

    logger.info("Creating dataloader.")
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []

    logger.info("Generating embeddings.")
    for i, batch_images in enumerate(tqdm(dataloader, unit_scale=BATCH_SIZE)):
        emb = embedder.embed(batch_images["image_transformed"]).cpu().numpy()
        # if i == 0:
        #     print(f"Shape of embedding for one batch: {emb.shape}")
        embeddings.append(emb)
    logger.info("Stacking embeddings.")
    embeddings = np.vstack(embeddings)

    return embeddings

## 6. Do memory profiling with one type of dataset

Results written near top of notebook

In [11]:
%load_ext memray

In [12]:
%%memray_flamegraph --native --follow-fork --temporal

USE_DAFT = True
ds_type = "daft_to_torch" if USE_DAFT else "plain_torch"

with TemporaryDirectory() as tmp:
    logger.info("Downloading test images.")
    dl_hf_images(dir=Path(tmp), max_images=NUM_TEST_IMAGES)
    imglob = tmp+"/*.png"
    logger.info("Starting embedding pipeline.")
    embeddings = compute_embeddings(model_name=MODEL_NAME,
                                    images_glob = imglob,
                                    batch_size=BATCH_SIZE,
                                    dataset_type=ds_type)
    logger.info("Done with embedding pipeline.")

2025-03-14 15:08:48.442 | INFO     | __main__:<module>:14 - Downloading test images.


Memray WARNING: Correcting symbol for malloc from 0x420620 to 0x7fceea689c60
Memray WARNING: Correcting symbol for free from 0x420ab0 to 0x7fceea68a370
100%|██████████| 256/256 [00:10<00:00, 24.24it/s]


Size of images on disk: 19.5 MB
2025-03-14 15:09:03.003 | INFO     | __main__:<module>:17 - Starting embedding pipeline.
2025-03-14 15:09:03.004 | INFO     | __main__:compute_embeddings:8 - Instantiating embedding model.
2025-03-14 15:09:05.323 | INFO     | __main__:compute_embeddings:11 - Creating dataset of type daft_to_torch.
2025-03-14 15:09:05.393 | INFO     | __main__:compute_embeddings:19 - Creating dataloader.
2025-03-14 15:09:05.395 | INFO     | __main__:compute_embeddings:24 - Generating embeddings.


0it [00:00, ?it/s]

🗡️ 🐟 Project: 00:00 

...

2025-03-14 15:11:12.329 | INFO     | __main__:compute_embeddings:30 - Stacking embeddings.
2025-03-14 15:11:12.334 | INFO     | __main__:<module>:22 - Done with embedding pipeline.


Output()

⚠  No debug information was found for the Python interpreter  ⚠

Without debug information reports showing native traces may not include file names and line numbers. Please use an 
interpreter built with debug symbols for best results. Check https://bloomberg.github.io/memray/native_mode.html 
for more information regarding how memray resolves symbols.

Output()

Results saved to /home/willsa/git/bedmap-dev/nbs/daft-try/memray-results/tmp92gkp7jy/flamegraph.html